In [ ]:
#Download of some libraries that could help us
!pip install kaggle
!pip install visualkeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d amerii/spacenet-7-multitemporal-urban-development

spacenet-7-multitemporal-urban-development.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Checking of the zip file with the dataset in the folder we are currently in
!ls

files	     model_plot.png  spacenet-7-multitemporal-urban-development.zip
kaggle.json  sample_data


In [ ]:
import zipfile

#Decompress the dataset file
zip_ref = zipfile.ZipFile('spacenet-7-multitemporal-urban-development.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

KeyboardInterrupt: ignored

In [ ]:
!pip install geopandas
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import rasterio as rio
from rasterio import features
from pathlib import Path
import pathlib
import geopandas as gpd
from descartes import PolygonPatch
from PIL import Image
import itertools
import re

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
mpl.rcParams['figure.dpi'] = 300 #increase plot resolution
mpl.rcParams['figure.dpi']

300.0

In [ ]:
def rasterize_geojson(geojson_path,reference_raster_path):
    gdf = gpd.read_file(geojson_path)
    done=True
    with rio.open(reference_raster_path) as raster:
        r = raster.read(1)
        if np.sum(r)>0:
          mask = image = features.rasterize(((polygon, 255) for polygon in gdf['geometry']),out_shape=r.shape)
        else:
          mask=None
          done=False
        
    return mask,done

In [ ]:
import os
from os import listdir
from os.path import isfile, join

TRAIN_PATH='/content/files/SN7_buildings_train/train/'
TEST_PATH='/content/files/SN7_buildings_test_public/test_public/'

#List with ids of every patient
train_name_folders=listdir(TRAIN_PATH)
test_name_folders=listdir(TEST_PATH)

#Checking of the number of ids and list of ids
print(len(train_name_folders))
print(train_name_folders)

print(len(test_name_folders))
print(test_name_folders)

60
['L15-1481E-1119N_5927_3715_13', 'L15-0924E-1108N_3699_3757_13', 'L15-1672E-1207N_6691_3363_13', 'L15-0977E-1187N_3911_3441_13', 'L15-1172E-1306N_4688_2967_13', 'L15-1669E-1153N_6678_3579_13', 'L15-1439E-1134N_5759_3655_13', 'L15-0434E-1218N_1736_3318_13', 'L15-1276E-1107N_5105_3761_13', 'L15-1617E-1207N_6468_3360_13', 'L15-0632E-0892N_2528_4620_13', 'L15-1049E-1370N_4196_2710_13', 'L15-0683E-1006N_2732_4164_13', 'L15-1298E-1322N_5193_2903_13', 'L15-1185E-0935N_4742_4450_13', 'L15-1748E-1247N_6993_3202_13', 'L15-1716E-1211N_6864_3345_13', 'L15-1703E-1219N_6813_3313_13', 'L15-0387E-1276N_1549_3087_13', 'L15-0760E-0887N_3041_4643_13', 'L15-1210E-1025N_4840_4088_13', 'L15-1389E-1284N_5557_3054_13', 'L15-1848E-0793N_7394_5018_13', 'L15-0614E-0946N_2459_4406_13', 'L15-1025E-1366N_4102_2726_13', 'L15-1691E-1211N_6764_3347_13', 'L15-1690E-1211N_6763_3346_13', 'L15-0577E-1243N_2309_3217_13', 'L15-1014E-1375N_4056_2688_13', 'L15-1204E-1202N_4816_3380_13', 'L15-0595E-1278N_2383_3079_13', 'L15

In [ ]:
import random
import numpy as np
import tensorflow as tf

#Seed setting for the reproducibility of the proof of concept
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

In [ ]:
import numpy as np
import cv2
import re

X_train=[]   #Images list
y_train=[]   #Masks list
IMG_SIZE=500

print('--------------DATASET DE ENTRENAMIENTO---------------')
for index,k in enumerate(train_name_folders):
  if index==31:
    break
  print(k)
  image_names = [f for f in listdir(TRAIN_PATH+k+'/images/') if isfile(join(TRAIN_PATH+k+'/images/', f))]
  image_names=sorted(image_names)
  print(str(index/len(train_name_folders)*100)+'%')
  print(str(len(image_names))+' images.\n')
  for i in image_names:
    image=cv2.imread(TRAIN_PATH+k+'/images/'+i)
    raster_path=TRAIN_PATH+k+'/images_masked/'+i
    geojson_path=TRAIN_PATH+k+'/labels_match_pix/'+i[:-4]+'_Buildings.geojson'
    mask,done=rasterize_geojson(geojson_path,raster_path)
    if done:
      X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
      y_train.append(((cv2.resize(mask,(IMG_SIZE, IMG_SIZE))>0)*1))

  '''
  mask_names = sorted([s for s in image_names if "mask" in s])
  image_names = sorted([s for s in image_names if not "mask" in s])
  mask_names.sort(key=lambda x: int(x.split('_')[4]))
  image_names.sort(key=lambda x: int(x.split('_')[4][:-4]))
  for img1,img2 in zip(image_names,mask_names):
    mri_image=cv2.imread(DATASET_FOLDER_PATH+k+'/'+img1)
    mask_image=cv2.imread(DATASET_FOLDER_PATH+k+'/'+img2)
    if np.sum(mask_image)>0:    #PONER CONDICION
      X_test.append(cv2.resize(mri_image,(IMG_SIZE, IMG_SIZE)))
      y_test.append((cv2.resize(mask_image,(IMG_SIZE, IMG_SIZE))[:,:,0]>0)*1)
  '''

--------------DATASET DE ENTRENAMIENTO---------------
L15-1481E-1119N_5927_3715_13
0.0%
24 images.

L15-0924E-1108N_3699_3757_13
1.6666666666666667%
23 images.

L15-1672E-1207N_6691_3363_13
3.3333333333333335%
24 images.

L15-0977E-1187N_3911_3441_13
5.0%
25 images.

L15-1172E-1306N_4688_2967_13
6.666666666666667%
24 images.

L15-1669E-1153N_6678_3579_13
8.333333333333332%
18 images.

L15-1439E-1134N_5759_3655_13
10.0%
18 images.

L15-0434E-1218N_1736_3318_13
11.666666666666666%
25 images.

L15-1276E-1107N_5105_3761_13
13.333333333333334%
25 images.

L15-1617E-1207N_6468_3360_13
15.0%
23 images.

L15-0632E-0892N_2528_4620_13
16.666666666666664%
25 images.

L15-1049E-1370N_4196_2710_13
18.333333333333332%
25 images.

L15-0683E-1006N_2732_4164_13
20.0%
22 images.

L15-1298E-1322N_5193_2903_13
21.666666666666668%
24 images.

L15-1185E-0935N_4742_4450_13
23.333333333333332%
24 images.

L15-1748E-1247N_6993_3202_13
25.0%
25 images.

L15-1716E-1211N_6864_3345_13
26.666666666666668%
24 images

ERROR:fiona._env:At line 74, character 51: Unterminated string
ERROR:fiona._env:Failed to read GeoJSON data


DriverError: ignored

In [ ]:
X_test=[]
y_test=[]

print('\n--------------DATASET DE TEST---------------')
for index,k in enumerate(test_name_folders):
  print(k)
  image_names = [f for f in listdir(TEST_PATH+k+'/images_masked/') if isfile(join(TEST_PATH+k+'/images_masked/', f))]
  image_names=sorted(image_names)
  print(str(index/len(train_name_folders)*100)+'%')
  print(str(len(image_names))+' images.\n')
  for i in image_names:
    image=cv2.imread(TEST_PATH+k+'/images/'+i)
    if not np.shape(image)==():
      #X_test.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))

In [ ]:
def two_plot(img1,img2):
  fig, (ax1, ax2) = plt.subplots(1, 2)
  ax1.imshow(img1)
  ax2.imshow(img2)
  plt.show()

In [ ]:
for ind in range(0,60):
  two_plot(X_train[ind],y_train[ind])

#Training

In [ ]:
import tensorflow as tf
import keras

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
init='he_normal'
act='relu'    #tf.keras.layers.LeakyReLU()

def unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS,init,act):
  #Build the model
  inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
  s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=act, kernel_initializer=init, padding='same')(s)
  c1 = tf.keras.layers.Dropout(0.2)(c1)
  c1 = tf.keras.layers.Conv2D(16, (3, 3), activation=act, kernel_initializer=init, padding='same')(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=act, kernel_initializer=init, padding='same')(p1)
  c2 = tf.keras.layers.Dropout(0.2)(c2)
  c2 = tf.keras.layers.Conv2D(32, (3, 3), activation=act, kernel_initializer=init, padding='same')(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=act, kernel_initializer=init, padding='same')(p2)
  c3 = tf.keras.layers.Dropout(0.2)(c3)
  c3 = tf.keras.layers.Conv2D(64, (3, 3), activation=act, kernel_initializer=init, padding='same')(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=act, kernel_initializer=init, padding='same')(p3)
  c4 = tf.keras.layers.Dropout(0.2)(c4)
  c4 = tf.keras.layers.Conv2D(128, (3, 3), activation=act, kernel_initializer=init, padding='same')(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
  
  c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=act, kernel_initializer=init, padding='same')(p4)
  c5 = tf.keras.layers.Dropout(0.3)(c5)
  c5 = tf.keras.layers.Conv2D(256, (3, 3), activation=act, kernel_initializer=init, padding='same')(c5)

  #Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, c4])
  c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=act, kernel_initializer=init, padding='same')(u6)
  c6 = tf.keras.layers.Dropout(0.2)(c6)
  c6 = tf.keras.layers.Conv2D(128, (3, 3), activation=act, kernel_initializer=init, padding='same')(c6)
  
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c3])
  c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=act, kernel_initializer=init, padding='same')(u7)
  c7 = tf.keras.layers.Dropout(0.2)(c7)
  c7 = tf.keras.layers.Conv2D(64, (3, 3), activation=act, kernel_initializer=init, padding='same')(c7)
  
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c2])
  c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=act, kernel_initializer=init, padding='same')(u8)
  c8 = tf.keras.layers.Dropout(0.2)(c8)
  c8 = tf.keras.layers.Conv2D(32, (3, 3), activation=act, kernel_initializer=init, padding='same')(c8)
  
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
  c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=act, kernel_initializer=init, padding='same')(u9)
  c9 = tf.keras.layers.Dropout(0.2)(c9)
  c9 = tf.keras.layers.Conv2D(16, (3, 3), activation=act, kernel_initializer=init, padding='same')(c9)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  return model

model=unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS,init,act)

In [ ]:
#Plotting of the architecture of our model
keras.utils.vis_utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

In [ ]:
import visualkeras

#Visualization of a diagram of the architecture of the model
visualkeras.layered_view(model, scale_xy=4, scale_z=0.5, legend=True,max_xy=1000, max_z=1000)

In [ ]:
from keras import backend as K

#Function in which the Jaccard metric is defined
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

In [ ]:
#Now, we set the model's loss with the Jaccard metric 
set_seed(0)
model=unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS,init,act=tf.keras.layers.LeakyReLU(alpha=0.1))
model.compile(optimizer = 'adam', loss = [jacard_coef_loss], metrics = [jacard_coef,'accuracy',])

In [ ]:
import numpy as np

#Arrays in float64 for the traing of the model with the Jaccard loss
X_train=np.array(X_train).astype('float64')
y_train=np.array(y_train).astype('float64')

In [ ]:
#Training of the model
callbacks = [tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_jacard_coef',mode='max'),tf.keras.callbacks.TensorBoard(log_dir='logs')]

results_2 = model.fit(np.array(X_train), np.array(y_train), validation_split=0.1, batch_size=16, epochs=200, callbacks=callbacks)

In [ ]:
#Display of the accuracy of the model over the training epochs
plt.plot(results_2.history['accuracy'],label='accuracy')
plt.plot(results_2.history['val_accuracy'],label='val_accuracy')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Display of the Jaccard coeficient of the model over the training epochs
plt.plot(results_2.history['jacard_coef'],label='jacard_coef')
plt.plot(results_2.history['val_jacard_coef'],label='val_jacard_coef')

plt.xlabel('Epochs')
plt.ylabel('Jacard_coef')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Display of the loss of the model over the training epochs
plt.plot(results_2.history['loss'],label='loss')
plt.plot(results_2.history['val_loss'],label='val_loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()